In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim            #提供optimizer 
import torch.utils.data as Data

import torchvision
import torchvision.transforms as transforms


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

torch.set_printoptions(linewidth=120)            #display options for output

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()           # 输入x:(batch,3,32,32)
        
        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),    # -> (batch,32,32,32)
            nn.BatchNorm2d(32),              #只有block第一个conv2d用了normalize
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),     # -> (batch,64,32,32)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),              #只有block最后进行了池化      # -> (batch,64,16,16)

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),   # -> (batch,128,16,16)
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),   # -> (batch,128,16,16)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),                                    # -> (batch,128,8,8)
            nn.Dropout2d(p=0.5),                         #只有第二个block用了dropout

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),  # -> (batch,256,8,8)
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),   # -> (batch,256,8,8)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),                                  # -> (batch,256,4,4)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):                    #一直在对一个输入的数据t做操作          
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)           # x从(b,256,4,4)变成了(b,4096)
        
        # fc layer
        x = self.fc_layer(x)

        return x

In [3]:
# 超参数
BATCH_SIZE = 100
EPOCH = 80
LR = 0.001


#network模型用GPU还是CPU训练,这里由于之前存入的network是CPU训练并保存的，所以这里用GPU会出问题
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,                                              
    download=False,
    transform=transforms.Compose([                 
        transforms.RandomHorizontalFlip(),     #这句代码主要是用来做数据增强的，为了防止训练出现过拟合，
                                               #通常在小型数据集上，通过随机翻转图片，随机调整图片的亮度，来达到增加训练时数据集的容量。
        transforms.RandomGrayscale(),          #这句代码也是做数据增强的，依概率转化为灰度图，默认概率p=0.5
        transforms.ToTensor(),                            #表示要将数据变成tensor
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),          #标准化
    ])
)
train_loader = torch.utils.data.DataLoader(  
    train_set                                                #表示提取的数据集是train_set
    ,batch_size=BATCH_SIZE                                        #表示批量大小为10,默认为0
    ,shuffle=True                                            #表示打乱数据顺序
    ,num_workers=2
)

print(train_set)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train


In [5]:
# prediction function
def pred_rate(preds,labels):
    return preds.eq(labels).sum().item()/labels.shape[0]

# For updating learning rate, 设置lr逐渐变小，最后converge
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [6]:
# 加载之前保存的net
network = Network().to(device)
#torch.cuda.set_device(0)
#import torch.backends.cudnn as cudnn
#cudnn.benchmark = False

In [7]:
optimizer=optim.Adam(network.parameters(),lr=LR) #需要将network的参数(包括weight)传入才能构建optimizer,并且learning rate=0.001
loss_func = nn.CrossEntropyLoss()

curr_lr = LR

for epoch in range(EPOCH):
    for step,(batch_x,batch_y) in enumerate(train_loader):
        batch_x,batch_y=batch_x.to(device),batch_y.to(device)     #如果可以，tensor放到gpu上
        
        out=network(batch_x)
        loss=loss_func(out,batch_y)            #F.cross_entropy函数计算loss
        
        optimizer.zero_grad()
        loss.backward()                         #使用backpropogation算法计算grad
        optimizer.step()    #表示将要用该optimizer，通过loss function的最小化来更新weights
        
        preds=out.argmax(dim=1)
        
        if (step+1)%100 == 0:
             print('Epoch: ', epoch, '| Step: ', step+1,'| loss: ',loss.item(), '| prection rate: ',pred_rate(preds,batch_y))
    
    # Decay learning rate，每20个epoch的时候更新一次lr
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 2.00 GiB total capacity; 234.23 MiB already allocated; 1.29 GiB free; 55.77 MiB cached)

In [32]:
torch.save(network,'cnn.pkl')

In [33]:
test_set = torchvision.datasets.CIFAR10(
    root='./data'                      
    ,train=False                                           
    ,download=False                                       
    ,transform=transforms.Compose([                 
        transforms.ToTensor(),                            #表示要将数据变成tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),          #标准化,数据变到了（-1，1）之间的标准值
    ])
)

test_loader = torch.utils.data.DataLoader(  
    test_set                                                #表示提取的数据集是train_set
    ,batch_size=BATCH_SIZE                                        #表示批量大小为10,默认为0
    ,shuffle=True                                            #表示打乱数据顺序
    ,num_workers=2
)

In [51]:
# test the model
# test,发现在测试集上表现不好，则这个模型有点过拟合了，可以再加入预训练，数据增强，需要更多的数据，或者更少的参数
network=torch.load('cnn.pkl')
network.eval()
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with torch.no_grad():         # 使用 torch,no_grad()构建不需要track的上下文环境，这个时候再不会跟踪track各个tensor的梯度
    sum_correct = 0
    total = 0
    for test_x,test_y in test_loader:               #test_loader里面要一个一个数据取，用for
        test_x,test_y=test_x.to(device),test_y.to(device)
        
        out=network(test_x)
        pred=out.argmax(dim=1)
        
        total += test_y.size(0)
        sum_correct += (pred.cpu().data.numpy()==test_y.cpu().data.numpy()).sum().item()
        
    my_rate=sum_correct/total
    print('By using the test set,the prediction rate is: ',my_rate)

By using the test set,the prediction rate is:  0.5937


In [ ]:
# label
labels=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [135]:
# load data，另一种不从torchvision.datasets中读取数据集，而是直接通过函数读取数据的方法 
# (但是这样就不能用torchvision自带的数据增强和标准化)

def load_CIFAR_batch(filename):
    with open(filename, 'rb')as f:
        datadict = pickle.load(f,encoding='latin1')
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32)
        Y = np.array(Y)
        return X, Y

def unpickle_labels(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#加载标签名字
labels = unpickle_labels("./data/batches.meta")
print(labels[b'label_names'])

#加载训练集
train_X=[]
train_Y=[]
for i in range(5):
    s = './data/data_batch_' + str(i+1)
    train_X1, train_Y1 = load_CIFAR_batch("./data/data_batch_1")
    train_X.append(train_X1)
    train_Y.append(train_Y1)
    
train_X = np.concatenate((train_X[0],train_X[1],train_X[2],train_X[3],train_X[4]),axis=0)
train_Y = np.concatenate((train_Y[0],train_Y[1],train_Y[2],train_Y[3],train_Y[4]),axis=0)   

#加载验证集
test_X,test_Y = load_CIFAR_batch('./data/test_batch')

print(train_X.shape)
print(train_Y.shape)

# 我现在的电脑内存不够，先尝试减少训练集，并且随机化输入
indices = np.random.choice(50000,50000)   #从50000个数据中抽取50000个数据
train_X = train_X[indices]   
train_Y = train_Y[indices]

print(train_X.shape)
print(train_Y.shape)


# 用这个方法读取数据的验证集的验证流程：
pred_Y = network(test_X.to(device))
pred_Y = np.argmax(pred_Y.cpu().data.numpy(),axis=1)

print(pred_Y.shape)

accurate = (pred_Y== test_Y.cpu().data.numpy()).sum().item()/test_Y.shape[0]
print(accurate)
print(pred_Y[:100])
print(test_Y[:100])

#print(test_Y[:10])
#pred_Y[:10]

[b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']
(50000, 3, 32, 32)
(50000,)
(50000, 3, 32, 32)
(50000,)
